In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("C:/Users/Lenovo/Downloads/Assignment2_q2_dataset.csv")
data.head()

,id,Date,number of bedrooms,number of bathrooms,living area,lot area,number of floors,waterfront present,number of views,condition of the house,...,Built Year,Renovation Year,Postal Code,Lattitude,Longitude,living_area_renov,lot_area_renov,Number of schools nearby,Distance from the airport,Price
0,6762810145,42491,5,2.50,3650,9050,2.0,0,4,5,...,1921,0,122003,52.8645,-114.557,2880,5400,2,58,2380000
1,6762810635,42491,4,2.50,2920,4000,1.5,0,0,5,...,1909,0,122004,52.8878,-114.470,2470,4000,2,51,1400000
2,6762810998,42491,5,2.75,2910,9480,1.5,0,0,3,...,1939,0,122004,52.8852,-114.468,2940,6600,1,53,1200000
3,6762812605,42491,4,2.50,3310,42998,2.0,0,0,3,...,2001,0,122005,52.9532,-114.321,3350,42847,3,76,838000
4,6762812919,42491,3,2.00,2710,4500,1.5,0,0,4,...,1929,0,122006,52.9047,-114.485,2060,4500,1,51,805000


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14620 entries, 0 to 14619
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     14620 non-null  int64  
 1   Date                                   14620 non-null  int64  
 2   number of bedrooms                     14620 non-null  int64  
 3   number of bathrooms                    14620 non-null  float64
 4   living area                            14620 non-null  int64  
 5   lot area                               14620 non-null  int64  
 6   number of floors                       14620 non-null  float64
 7   waterfront present                     14620 non-null  int64  
 8   number of views                        14620 non-null  int64  
 9   condition of the house                 14620 non-null  int64  
 10  grade of the house                     14620 non-null  int64  
 11  Ar

In [4]:
data = data.drop(['id', 'Date'], axis=1)

In [5]:
X = data.drop('Price', axis=1).values
y = data['Price'].values.reshape(-1, 1)

In [6]:
X_mean, X_std = np.mean(X, axis=0), np.std(X, axis=0)
X = (X - X_mean) / X_std

In [7]:
y_mean, y_std = np.mean(y), np.std(y)
y = (y - y_mean) / y_std

In [8]:
split_ratio = 0.8
split_index = int(split_ratio * len(X))

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [9]:
input_size = X_train.shape[1]
hidden_size = 32
output_size = 1
learning_rate = 0.0001
epochs = 1000

In [10]:
weights_1 = np.random.randn(input_size, hidden_size) * 0.01
biases_1 = np.zeros((1, hidden_size))
weights_2 = np.random.randn(hidden_size, output_size) * 0.01
biases_2 = np.zeros((1, output_size))

In [11]:
def relu(x):
    return np.maximum(0, x)

def linear_activation(x):
    return x

In [12]:
for epoch in range(epochs):
    
    hidden_layer_input = np.dot(X_train, weights_1) + biases_1
    hidden_layer_output = relu(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_2) + biases_2
    predicted_output = linear_activation(output_layer_input)

    
    loss = np.mean((predicted_output - y_train) ** 2)

    
    output_error = predicted_output - y_train
    hidden_layer_error = np.dot(output_error, weights_2.T)
    hidden_layer_error[hidden_layer_input <= 0] = 0  # ReLU derivative

    
    weights_2 -= learning_rate * np.dot(hidden_layer_output.T, output_error)
    biases_2 -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
    weights_1 -= learning_rate * np.dot(X_train.T, hidden_layer_error)
    biases_1 -= learning_rate * np.sum(hidden_layer_error, axis=0, keepdims=True)

    
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, Loss: {loss}")

Epoch: 0, Loss: 1.0170114839846125
Epoch: 100, Loss: 0.942734143556738
Epoch: 200, Loss: 0.32622428823519256
Epoch: 300, Loss: 0.44450730914590253
Epoch: 400, Loss: 0.4364215851433712
Epoch: 500, Loss: 0.4323365373618606
Epoch: 600, Loss: 0.4271392472677736
Epoch: 700, Loss: 0.41812221281325024
Epoch: 800, Loss: 0.401035160562391
Epoch: 900, Loss: 0.39121389141824503


In [13]:
hidden_layer_test = relu(np.dot(X_test, weights_1) + biases_1)
predicted_test = linear_activation(np.dot(hidden_layer_test, weights_2) + biases_2)

predicted_test = (predicted_test * y_std) + y_mean


mse = np.mean((predicted_test - data['Price'][split_index:].values) ** 2)
print(f"Mean Squared Error on Test Set: {mse}")

Mean Squared Error on Test Set: 276237405218.0545


In [14]:
from sklearn.metrics import r2_score

In [15]:
print(r2_score(predicted_test , data['Price'][split_index:].values))

0.5574478201465385


In [16]:
first_row = data.iloc[0]
x = first_row[:-1]

In [18]:
sample_input = np.array(x)


sample_input_normalized = (sample_input - X_mean) / X_std


hidden_layer_input_sample = np.dot(sample_input_normalized, weights_1) + biases_1
hidden_layer_output_sample = relu(hidden_layer_input_sample)
output_layer_input_sample = np.dot(hidden_layer_output_sample, weights_2) + biases_2
predicted_price_sample = linear_activation(output_layer_input_sample)


predicted_price_sample_denormalized = (predicted_price_sample * y_std) + y_mean

print("Predicted Price for the Sample Input:", predicted_price_sample_denormalized[0, 0])


Predicted Price for the Sample Input: 2279187.301355173
